In [7]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.utils.ultralytics as fou
from ultralytics import YOLO
from fiftyone import ViewField as F 


In [54]:
# Create a new dataset and import the data in Pascal VOC format in one step
dataset = fo.Dataset.from_dir(
    dataset_dir='Images',
    dataset_type=fo.types.VOCDetectionDataset,
    name="cctv_guns_hackathon"
)


ValueError: Dataset name 'cctv_guns_hackathon' is not available

In [4]:
# Use an existing dataset in another session:
dataset = fo.load_dataset("cctv_guns_hackathon_new")


In [5]:
# Launch the FiftyOne App to visualize your dataset
session = fo.launch_app(dataset)


In [30]:
session.close()


ConnectionError: HTTPConnectionPool(host='localhost', port=5151): Max retries exceeded with url: /event (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x318a63020>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [9]:
# value labels
classes_labels = dataset.values("ground_truth.detections.label")
print(classes_labels)


[[], ['Handgun'], ['Handgun', 'Short_rifle'], ['Short_rifle', 'Handgun'], ['Short_rifle', 'Handgun'], ['Short_rifle'], ['Short_rifle'], ['Short_rifle'], ['Short_rifle'], ['Short_rifle'], ['Short_rifle'], ['Short_rifle'], ['Knife', 'Handgun'], ['Short_rifle'], ['Short_rifle'], ['Short_rifle'], ['Short_rifle'], ['Short_rifle'], ['Short_rifle'], ['Knife', 'Handgun', 'Handgun'], ['Knife', 'Handgun'], ['Handgun', 'Knife', 'Handgun'], ['Knife', 'Handgun', 'Handgun'], ['Knife', 'Handgun', 'Handgun'], ['Handgun', 'Handgun'], ['Handgun', 'Handgun'], ['Handgun', 'Handgun'], [], ['Handgun', 'Handgun'], ['Handgun', 'Handgun'], ['Handgun', 'Handgun'], ['Handgun', 'Handgun'], ['Handgun', 'Handgun'], ['Handgun', 'Handgun', 'Knife'], ['Handgun', 'Handgun', 'Knife'], ['Handgun', 'Handgun'], ['Handgun'], ['Handgun', 'Handgun'], [], ['Handgun', 'Handgun'], ['Handgun', 'Handgun'], ['Handgun', 'Handgun'], ['Handgun', 'Handgun'], ['Handgun', 'Handgun'], ['Handgun', 'Handgun'], ['Handgun', 'Handgun'], ['Hand

In [ ]:
print(dataset)


Name:        cctv_guns_hackathon_new
Media type:  image
Num samples: 5149
Persistent:  False
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)


In [15]:
# Get the unique classes
classes = ["Handgun", "Knife", "Short_rifle"]
print(f"Using classes: {classes}")

# Count samples with and without detections
samples_with_detections = dataset.match(F("ground_truth.detections").length() > 0)
samples_without_detections = dataset.match(F("ground_truth.detections").length() == 0)

print(f"Samples with detections: {samples_with_detections.count()}")
print(f"Samples without detections: {samples_without_detections.count()}")

# Randomly reduce no-detection samples to match the count of detection samples
balanced_no_detection_samples = samples_without_detections.take(samples_with_detections.count())

# Combine detection and balanced no-detection samples
balanced_dataset = samples_with_detections.clone()
balanced_dataset.add_samples(balanced_no_detection_samples)

print(f"Balanced dataset size: {balanced_dataset.count()}")


Using classes: ['Handgun', 'Knife', 'Short_rifle']
Samples with detections: 1534
Samples without detections: 3615
 100% |███████████████| 1534/1534 [621.0ms elapsed, 0s remaining, 2.5K samples/s]      
Balanced dataset size: 3068


In [16]:
# Shuffle the dataset to ensure a random distribution of samples
shuffled_dataset = balanced_dataset.shuffle(seed=42)  # Set seed for reproducibility

# Define split sizes (80% train, 10% val, 10% test)
total_samples = len(shuffled_dataset)
train_size = int(0.8 * total_samples)
val_size = int(0.1 * total_samples)
test_size = total_samples - train_size - val_size  # Remaining for test

# Get the splits after shuffling
train_view = shuffled_dataset.take(train_size)  # Take the first 80% randomly
val_view = shuffled_dataset.skip(train_size).take(val_size)  # Take the next 10%
test_view = shuffled_dataset.skip(train_size + val_size).take(test_size)  # Remaining samples

# Print counts for verification
print(f"Total samples: {total_samples}")
print(f"Train samples: {len(train_view)}")
print(f"Validation samples: {len(val_view)}")
print(f"Test samples: {len(test_view)}")


Total samples: 3068
Train samples: 2454
Validation samples: 306
Test samples: 308


In [17]:
# Set the export directory
EXPORT_DIR = "./yolo"  

# Export training split
train_view.export(
    export_dir=EXPORT_DIR,
    dataset_type=fo.types.YOLOv5Dataset,  # Ensure YOLOv5 dataset type is correct
    label_field="ground_truth",           # Field that contains the labels
    split="train",                        # Specify the split as 'train'
    classes=["Handgun", "Knife", "Short_rifle"]  # Define your classes
)
print("Training data export complete")


Directory './yolo' already exists; export will be merged with existing files
 100% |███████████████| 2454/2454 [3.8s elapsed, 0s remaining, 660.0 samples/s]      
Training data export complete


In [18]:
# Export validation data
print("Exporting validation data...")
val_view.export(
    export_dir=EXPORT_DIR,
    dataset_type=fo.types.YOLOv5Dataset,
    label_field="ground_truth",
    split="val",
    classes=["Handgun", "Knife", "Short_rifle"]
)
print("Validation data export complete")


Exporting validation data...
Directory './yolo' already exists; export will be merged with existing files
 100% |█████████████████| 306/306 [613.4ms elapsed, 0s remaining, 498.9 samples/s]      
Validation data export complete


In [19]:
# Export test data
print("Exporting test data...")
test_view.export(
    export_dir=EXPORT_DIR,
    dataset_type=fo.types.YOLOv5Dataset,
    label_field="ground_truth",
    split="test",
    classes=["Handgun", "Knife", "Short_rifle"]
)
print("Test data export complete")


Exporting test data...
Directory './yolo' already exists; export will be merged with existing files
 100% |█████████████████| 308/308 [378.0ms elapsed, 0s remaining, 814.9 samples/s]      
Test data export complete


In [24]:
from ultralytics import YOLO

# Load pretrained YOLO11 model
model = YOLO('yolo11n.pt')  # YOLO11 nano model

# Train with improved settings
model.train(
    data="./yolo/dataset.yaml",
    epochs=50,               # Increase epochs substantially
    imgsz=320,               # Larger image size for better detection
    batch=128,                # Increase if your GPU has enough memory
    patience=10,             # Early stopping if no improvement after 10 epochs
    optimizer='AdamW'        # Use AdamW optimizer explicitly
)

# Evaluate performance
metrics = model.val()
print(metrics)


Ultralytics 8.3.102 🚀 Python-3.13.2 torch-2.6.0 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=./yolo/dataset.yaml, epochs=50, time=None, patience=10, batch=128, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, 

train: Scanning /Users/laurisoome/Desktop/hackathon-gun-detection/yolo/labels/train.cache... 3510 images, 2042 backgrounds, 0 corrupt: 100%|██████████| 3510/3510 [00:00<?, ?it/s]
val: Scanning /Users/laurisoome/Desktop/hackathon-gun-detection/yolo/labels/val.cache... 602 images, 442 backgrounds, 0 corrupt: 100%|██████████| 602/602 [00:00<?, ?it/s]

Plotting labels to /Users/laurisoome/Desktop/hackathon-gun-detection/runs/detect/train7/labels.jpg... 


optimizer: AdamW(lr=0.01, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.001), 87 bias(decay=0.0)
Image sizes 320 train, 320 val
Using 0 dataloader workers
Logging results to /Users/laurisoome/Desktop/hackathon-gun-detection/runs/detect/train7
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      4.129      6.385      1.251         55        320: 100%|██████████| 28/28 [13:25<00:00, 28.77s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:25<00:00,  8.47s/it]

...

       5/50         0G      3.836       3.21      1.158         63        320: 100%|██████████| 28/28 [11:58<00:00, 25.66s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:24<00:00,  8.33s/it]

                   all        602        284      0.346     0.0018   0.000475   0.000116



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       6/50         0G      3.865      3.207      1.141        152        320:  57%|█████▋    | 16/28 [08:15<06:11, 30.95s/it]


KeyboardInterrupt: 

In [31]:
# Load your existing dataset and model
dataset = fo.load_dataset("cctv_guns_hackathon_new")
model = YOLO("./models/yolov11_n_best.pt")

# Get validation images
val_view = dataset.match_tags("val")

# Run inference on validation images
for sample in val_view:
    # Run inference
    results = model(sample.filepath)
    result = results[0]
    
    # Extract predictions
    detections = []
    if result.boxes is not None:
        for box, conf, cls in zip(result.boxes.xyxy, result.boxes.conf, result.boxes.cls):
            x1, y1, x2, y2 = box.tolist()
            detection = fo.Detection(
                label=model.names[int(cls.item())],
                bounding_box=[
                    x1 / result.orig_shape[1],
                    y1 / result.orig_shape[0],
                    (x2 - x1) / result.orig_shape[1],
                    (y2 - y1) / result.orig_shape[0],
                ],
                confidence=conf.item()
            )
            detections.append(detection)
    
    # Add predictions to sample
    sample["predictions"] = fo.Detections(detections=detections)
    sample.save()

# Launch the app
session = fo.launch_app(dataset)
session.view = val_view.sort_by("predictions.detections.confidence", reverse=True)


ConnectionError: HTTPConnectionPool(host='localhost', port=5151): Max retries exceeded with url: /event (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x17ece98c0>: Failed to establish a new connection: [Errno 61] Connection refused'))


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds

